In [1]:
from collections import deque


class Automato:
    """Implementa um automato finito determinístico (AFD) que obedece a 
    seguinte definição:
    M = (Q, Σ, δ, q0, F)
    Q: Conjunto finito de estados
    Σ: Alfabeto
    δ: Função de transição
    q0: Estado inicial
    F: Conjunto de estados finai
    """
    
    estados: set[str]
    alfabeto: set[str]
    transicoes: dict[dict[str, str]]
    estado_inicial: str
    conj_estados_finais: set[str]
    fita: deque[str]

    def __init__(self, estados, alfabeto, transicoes,
                estado_inicial, conj_estados_finais):
        self.estados: set = estados
        self.alfabeto: set = alfabeto
        self.transicoes = transicoes
        self.estado_inicial = estado_inicial
        self.estados_finais = conj_estados_finais
        self.fita = deque()
        self.cursor = 0
        self.maquina_estados: MaquinaEstados = MaquinaEstados(self)

    def __repr__(self) -> str:
        return f"""\
        M = (Q, Σ, δ, q0, F)
        Q: {{{','.join(self.estados)}}}
        Σ: {{{self.alfabeto}}}
        δ: {self.maquina_estados.transicao}
        q0: {self.estado_inicial}
        F: {self.estados_finais}\
        """
    
    def escrever_fita(self, cadeia: str) -> None:
        """Escreve uma cadeia de caracteres na fita."""
        self.fita.clear()
        self.fita = deque(cadeia)

    def reconhecer_cadeia_alt(self, cadeia: str):
        """Reconhece uma cadeia de caracteres na fita usando uma
        estrutura do tipo fila (deque).
        """
        self.maquina_estados.resetar()    
        self.escrever_fita(cadeia)
        self.maquina_estados.gravar_movimento(self.fita)
        while self.fita:
            simbolo = self.fita.popleft()
            try:
                self.maquina_estados.config_seguinte(simbolo)
                self.maquina_estados.gravar_movimento(self.fita)
            except KeyError:
                return False
        estado_parada = self.maquina_estados.estado_atual
        return bool(estado_parada in self.estados_finais)
    

# Maquina de estados
class MaquinaEstados:
    """Classe que implementa uma máquina de estados finitos 
    determinística por composição da classe Automato.
    """
    
    funcao_transicao: dict[dict[str, str]]
    estado_inicial: str
    estados_finais: set[str]


    def __init__(self, automato: Automato):
        self.transicao = automato.transicoes
        self.estados_finais = automato.estados_finais
        self.estado_inical = automato.estado_inicial
        self.estado_atual = automato.estado_inicial
        self.cadeia_restante = None
        self.posicao_cursor = 0
        self.movimentos = list()
    
    def config_seguinte(self, simbolo):
        """Aplica a função de transição δ(p, σ) -> q."""
        if simbolo != "\0":
            self.estado_atual = self.transicao[self.estado_atual][simbolo]
            self.posicao_cursor += 1

    def gravar_movimento(self, cadeia: deque[str]):
        """Retorna o histórico de movimentos da máquina de estados."""
        self.movimentos.append((self.estado_atual, "".join(cadeia)))

    def apresentar_movimentos(self):
        """Formata o conteúdo do histórico de movimentos da máquina de 
        estados usando o símbolo da catraca.
        """
        mov_formatados = str(self.movimentos).replace("), ", ") ⊢ ")
        return mov_formatados

    def resetar(self):
        """Reseta a máquina de estados."""
        self.estado_atual = self.estado_inical
        self.cadeia_restante = None
        self.posicao_cursor = 0
        self.movimentos = list()

    # Opcao com recursao
    def reconhecer_cadeia(self, cadeia):
        """Reconhece uma cadeia na fita usando recursão."""

        if self.cadeia_restante == None:
            self.movimentos = []
            self.movimentos.append((self.estado_atual, cadeia))
        
        resultado = False
        if (self.estado_atual, cadeia[0]) in self.transicao.keys():
        
            self.estado_atual = self.transicao[(self.estado_atual, cadeia[0])]
            self.cadeia_restante = cadeia[1:]
            self.movimentos.append((self.estado_atual, self.cadeia_restante))
            if self.cadeia_restante == "":
                return True if self.estado_atual in self.estados_finais else False
            else:
                resultado = self.reconhecer_cadeia(self.cadeia_restante)
        else:            
            return False
        self.estado_atual = self.estado_inical
        self.cadeia_restante = None
        return resultado

class Transdutores(Automato):
    def __init__(self, estados, alfabeto_entrada, alfabeto_saida, transicoes, transducoes, estado_inicial, conj_estados_finais):
        super().__init__(estados, alfabeto_entrada, transicoes, estado_inicial, conj_estados_finais)
        self.alfabeto_saida = alfabeto_saida
        self.transducoes = transducoes
        self._cadeia_saida = ""

    def reconhecer_cadeia(self, cadeia):
        """Reconhece uma cadeia na fita usando recursão."""

        if self.maquina_estados.cadeia_restante == None:
            self.movimentos = []
            self.movimentos.append((self.maquina_estados.estado_atual, cadeia))
            self.posicao_cursor = 0
            self._cadeia_saida = self.transducoes[self.maquina_estados.estado_atual]
        
        resultado = False
        if (self.maquina_estados.estado_atual, cadeia[0]) in self.maquina_estados.transicao.keys():
            
            self.maquina_estados.estado_atual = self.maquina_estados.transicao[(self.maquina_estados.estado_atual, cadeia[0])]
            self._cadeia_saida += self.transducoes[self.maquina_estados.estado_atual]
            self.posicao_cursor += 1
            self.maquina_estados.cadeia_restante = cadeia[1:]
            self.movimentos.append((self.maquina_estados.estado_atual, self.maquina_estados.cadeia_restante))
            if self.maquina_estados.cadeia_restante == "":
                return self._cadeia_saida if self.maquina_estados.estado_atual in self.estados_finais else False
            else:
                resultado = self.reconhecer_cadeia(self.maquina_estados.cadeia_restante)
        else:            
            return False
        self.maquina_estados.estado_atual = self.maquina_estados.estado_inical
        self.maquina_estados.cadeia_restante = None
        return resultado


In [2]:
# Automato de teste
estados = set(["q0", "q1", "q2", "q3"])
alfabeto = set(['0', '1', '2'])
funcao_de_transicao = {
    ('q0', '0'): 'q0',
    ('q0', '1'): 'q1',
    ('q0', '2'): 'q3',
    ('q1', '0'): 'q3',
    ('q1', '1'): 'q1',
    ('q1', '2'): 'q2',
    ('q2', '0'): 'q3',
    ('q2', '1'): 'q3',
    ('q2', '2'): 'q2',
    ('q3', '0'): 'q3',
    ('q3', '1'): 'q3',
    ('q3', '2'): 'q3',
}
estado_inicial = 'q0'
estados_finais = set(['q1', 'q2'])

In [3]:
automato1 = Automato(estados, alfabeto, funcao_de_transicao, estado_inicial, estados_finais)

In [4]:
# Automato de teste
estados = set(["q0", "q1", "q2", "q3"])
alfabeto_entrada = set(['0', '1', '2'])
alfabeto_saida = set(['1', '2'])
transducoes = {
    'q0': '1',
    'q1': '2',
    'q2': '1',
    'q3': '2',
}
funcao_de_transicao = {
    ('q0', '0'): 'q0',
    ('q0', '1'): 'q1',
    ('q0', '2'): 'q3',
    ('q1', '0'): 'q3',
    ('q1', '1'): 'q1',
    ('q1', '2'): 'q2',
    ('q2', '0'): 'q3',
    ('q2', '1'): 'q3',
    ('q2', '2'): 'q2',
    ('q3', '0'): 'q3',
    ('q3', '1'): 'q3',
    ('q3', '2'): 'q3',
}
estado_inicial = 'q0'
estados_finais = set(['q1', 'q2'])


In [5]:
transdutor1 = Transdutores(estados, alfabeto_entrada, alfabeto_saida, funcao_de_transicao, transducoes, estado_inicial, estados_finais)

In [6]:
print(transdutor1.maquina_estados.reconhecer_cadeia("012"))
print(transdutor1.reconhecer_cadeia("012"))

True
1121


In [7]:
cadeia = ("25", "50", "100")
print("".join(cadeia))

2550100


In [9]:
from collections import defaultdict  

estados = set(["q0", "q1", "q2", "q3", "q4", "q5", "q6", "q7", "q8", "q9", "q10", "q11", "q12", "q13"])
diacriticos = set(['é', 'í', 'á', 'ó', 'ú', 'õ', 'ã', 'â', 'ê', 'ô', 'ç', 'à'])
alfabeto_entrada = set(set([chr(i) for i in range(32,128)]).union(diacriticos).union(set(map(str.upper, diacriticos))))

dicio_teste = defaultdict(lambda: "q0")
funcao_de_transicao_alt = defaultdict(lambda: "q1")
funcao_de_transicao_alt.update((
    (('q0', 'C'), 'q3'),
    (('q1', ' '), 'q2'),
    (('q2', 'C'), 'q3'),
    (('q2', 'c'), 'q3'),
    (('q2', ' '), 'q2'),
    (('q3', 'o'), 'q4'),
    (('q3', ' '), 'q2'),
    (('q4', 'm'), 'q5'),
    (('q4', ' '), 'q2'),
    (('q5', 'p'), 'q6'),
    (('q5', ' '), 'q2'),
    (('q6', 'u'), 'q7'),
    (('q6', ' '), 'q2'),
    (('q7', 't'), 'q8'),
    (('q7', ' '), 'q2'),
    (('q8', 'a'), 'q9'),
    (('q8', ' '), 'q2'),
    (('q9', 'd'), 'q10'),
    (('q9', ' '), 'q2'),
    (('q10', 'o'), 'q11'),
    (('q10', ' '), 'q2'),
    (('q11', 'r'), 'q12'),
    (('q11', ' '), 'q2'),
    (('q12', ','), 'q13'),
    (('q12', '?'), 'q13'),
    (('q12', ';'), 'q13'),
    (('q12', '!'), 'q13'),
    (('q12', ' '), 'q13'),
    (('q12', '.'), 'q13'),
    (('q12', ':'), 'q13'),
    (('q13', 'c'), 'q3'),
    (('q13', 'C'), 'q3'),
    (('q13', ' '), 'q2'))
)
for s in alfabeto_entrada:
    dicio_teste[("q0",s)]
    print(dicio_teste[("q0",s)], s)

q0 ,
q0 5
q0 B
q0 L
q0 j
q0 &
q0 \
q0 s
q0 d
q0 ^
q0 @
q0 ã
q0 Ó
q0 ç
q0 Ã
q0 +
q0 m
q0 Ô
q0 é
q0 1
q0 x
q0 I
q0 0
q0 q
q0 /
q0 e
q0 u
q0 O
q0 :
q0 g
q0 K
q0 [
q0 õ
q0 D
q0 2
q0 #
q0 6
q0 .
q0 !
q0 *
q0 p
q0 À
q0 b
q0 U
q0 y
q0 7
q0 í
q0 $
q0 r
q0 ó
q0 l
q0 <
q0 E
q0 t
q0 f
q0 z
q0 J
q0 i
q0 Y
q0 Ç
q0 M
q0 "
q0 ú
q0 v
q0 (
q0 R
q0 -
q0 X
q0 â
q0 N
q0 w
q0 É
q0 G
q0 á
q0 Á
q0 W
q0 Ú
q0  
q0 P
q0 ô
q0 A
q0 ]
q0 ?
q0 >
q0 H
q0 ê
q0 4
q0 ;
q0 _
q0 }
q0 '
q0 n
q0 à
q0 k
q0 Z
q0 9
q0 8
q0 Q
q0 a
q0 Õ
q0 Í
q0 F
q0 {
q0 o
q0 Ê
q0 Â
q0 h
q0 C
q0 
q0 `
q0 V
q0 =
q0 |
q0 T
q0 )
q0 c
q0 ~
q0 S
q0 3
q0 %


In [15]:
funcao_de_transicao_alt[('q0', 'k')]
for transicao in funcao_de_transicao_alt.items():
    print(transicao)

print(funcao_de_transicao_alt[('q0', 'X')])

(('q0', 'C'), 'q3')
(('q1', ' '), 'q2')
(('q2', 'C'), 'q3')
(('q2', 'c'), 'q3')
(('q2', ' '), 'q2')
(('q3', 'o'), 'q4')
(('q3', ' '), 'q2')
(('q4', 'm'), 'q5')
(('q4', ' '), 'q2')
(('q5', 'p'), 'q6')
(('q5', ' '), 'q2')
(('q6', 'u'), 'q7')
(('q6', ' '), 'q2')
(('q7', 't'), 'q8')
(('q7', ' '), 'q2')
(('q8', 'a'), 'q9')
(('q8', ' '), 'q2')
(('q9', 'd'), 'q10')
(('q9', ' '), 'q2')
(('q10', 'o'), 'q11')
(('q10', ' '), 'q2')
(('q11', 'r'), 'q12')
(('q11', ' '), 'q2')
(('q12', ','), 'q13')
(('q12', '?'), 'q13')
(('q12', ';'), 'q13')
(('q12', '!'), 'q13')
(('q12', ' '), 'q13')
(('q12', '.'), 'q13')
(('q12', ':'), 'q13')
(('q13', 'c'), 'q3')
(('q13', 'C'), 'q3')
(('q13', ' '), 'q2')
(('q0', 'k'), 'q1')
q1


In [ ]:
funcao_de_transicao = {
    ('q0', 'C'): 'q3', # Recebendo o C no inicio avança para q3
    ('q0', 'S'): 'q1',
    ('q0', 'ç'): 'q1',
    ('q0', ','): 'q1',
    ('q0', 'N'): 'q1',
    ('q0', 'm'): 'q1',
    ('q0', '&'): 'q1',
    ('q0', 'Í'): 'q1',
    ('q0', 'Ê'): 'q1',
    ('q0', 'D'): 'q1',
    ('q0', 'B'): 'q1',
    ('q0', 'à'): 'q1',
    ('q0', 'ô'): 'q1',
    ('q0', '"'): 'q1',
    ('q0', '{'): 'q1',
    ('q0', '%'): 'q1',
    ('q0', 'á'): 'q1',
    ('q0', 'À'): 'q1',
    ('q0', 'E'): 'q1',
    ('q0', '?'): 'q1',
    ('q0', 'Z'): 'q1',
    ('q0', 'x'): 'q1',
    ('q0', '7'): 'q1',
    ('q0', '4'): 'q1',
    ('q0', ']'): 'q1',
    ('q0', ';'): 'q1',
    ('q0', 'r'): 'q1',
    ('q0', '/'): 'q1',
    ('q0', '!'): 'q1',
    ('q0', 'P'): 'q1',
    ('q0', '='): 'q1',
    ('q0', ' '): 'q1',
    ('q0', 'í'): 'q1',
    ('q0', 'i'): 'q1',
    ('q0', 'ã'): 'q1',
    ('q0', 'u'): 'q1',
    ('q0', 'V'): 'q1',
    ('q0', '.'): 'q1',
    ('q0', '1'): 'q1',
    ('q0', "'"): 'q1',
    ('q0', 'v'): 'q1',
    ('q0', '2'): 'q1',
    ('q0', 'Y'): 'q1',
    ('q0', '#'): 'q1',
    ('q0', 'n'): 'q1',
    ('q0', ')'): 'q1',
    ('q0', 'w'): 'q1',
    ('q0', '>'): 'q1',
    ('q0', '<'): 'q1',
    ('q0', '_'): 'q1',
    ('q0', 'g'): 'q1',
    ('q0', 'c'): 'q1',
    ('q0', 'J'): 'q1',
    ('q0', '9'): 'q1',
    ('q0', 's'): 'q1',
    ('q0', 'ó'): 'q1',
    ('q0', 'é'): 'q1',
    ('q0', 'O'): 'q1',
    ('q0', 'Ô'): 'q1',
    ('q0', '0'): 'q1',
    ('q0', 'F'): 'q1',
    ('q0', 'M'): 'q1',
    ('q0', '`'): 'q1',
    ('q0', ''): 'q1',
    ('q0', 'Ã'): 'q1',
    ('q0', 'H'): 'q1',
    ('q0', 'õ'): 'q1',
    ('q0', '@'): 'q1',
    ('q0', 'U'): 'q1',
    ('q0', 'b'): 'q1',
    ('q0', '['): 'q1',
    ('q0', 'ê'): 'q1',
    ('q0', '5'): 'q1',
    ('q0', 'T'): 'q1',
    ('q0', '+'): 'q1',
    ('q0', '|'): 'q1',
    ('q0', 'q'): 'q1',
    ('q0', '$'): 'q1',
    ('q0', 'p'): 'q1',
    ('q0', '('): 'q1',
    ('q0', '6'): 'q1',
    ('q0', 'ú'): 'q1',
    ('q0', '3'): 'q1',
    ('q0', 'X'): 'q1',
    ('q0', 'W'): 'q1',
    ('q0', 'Ç'): 'q1',
    ('q0', 'G'): 'q1',
    ('q0', 'Q'): 'q1',
    ('q0', '-'): 'q1',
    ('q0', ':'): 'q1',
    ('q0', 'R'): 'q1',
    ('q0', 'k'): 'q1',
    ('q0', 'A'): 'q1',
    ('q0', '^'): 'q1',
    ('q0', '}'): 'q1',
    ('q0', '8'): 'q1',
    ('q0', 'Á'): 'q1',
    ('q0', 'e'): 'q1',
    ('q0', 'I'): 'q1',
    ('q0', 'a'): 'q1',
    ('q0', 'Ú'): 'q1',
    ('q0', 'Â'): 'q1',
    ('q0', 'É'): 'q1',
    ('q0', 't'): 'q1',
    ('q0', 'K'): 'q1',
    ('q0', 'â'): 'q1',
    ('q0', '*'): 'q1',
    ('q0', '~'): 'q1',
    ('q0', 'h'): 'q1',
    ('q0', 'l'): 'q1',
    ('q0', 'f'): 'q1',
    ('q0', '\\'): 'q1',
    ('q0', 'y'): 'q1',
    ('q0', 'j'): 'q1',
    ('q0', 'd'): 'q1',
    ('q0', 'L'): 'q1',
    ('q0', 'Õ'): 'q1',
    ('q0', 'Ó'): 'q1',
    ('q0', 'z'): 'q1',
    ('q0', 'o'): 'q1',
    ('q1', ' '): 'q2', #Recebendo o espaço avança para o estado q2
    ('q1', 'S'): 'q1',
    ('q1', 'ç'): 'q1',
    ('q1', ','): 'q1',
    ('q1', 'N'): 'q1',
    ('q1', 'm'): 'q1',
    ('q1', '&'): 'q1',
    ('q1', 'Í'): 'q1',
    ('q1', 'Ê'): 'q1',
    ('q1', 'D'): 'q1',
    ('q1', 'B'): 'q1',
    ('q1', 'à'): 'q1',
    ('q1', 'ô'): 'q1',
    ('q1', '"'): 'q1',
    ('q1', '{'): 'q1',
    ('q1', '%'): 'q1',
    ('q1', 'á'): 'q1',
    ('q1', 'À'): 'q1',
    ('q1', 'E'): 'q1',
    ('q1', '?'): 'q1',
    ('q1', 'Z'): 'q1',
    ('q1', 'x'): 'q1',
    ('q1', '7'): 'q1',
    ('q1', '4'): 'q1',
    ('q1', ']'): 'q1',
    ('q1', ';'): 'q1',
    ('q1', 'r'): 'q1',
    ('q1', '/'): 'q1',
    ('q1', '!'): 'q1',
    ('q1', 'P'): 'q1',
    ('q1', '='): 'q1',
    ('q1', 'í'): 'q1',
    ('q1', 'i'): 'q1',
    ('q1', 'ã'): 'q1',
    ('q1', 'u'): 'q1',
    ('q1', 'V'): 'q1',
    ('q1', '.'): 'q1',
    ('q1', '1'): 'q1',
    ('q1', "'"): 'q1',
    ('q1', 'v'): 'q1',
    ('q1', '2'): 'q1',
    ('q1', 'Y'): 'q1',
    ('q1', '#'): 'q1',
    ('q1', 'n'): 'q1',
    ('q1', ')'): 'q1',
    ('q1', 'w'): 'q1',
    ('q1', '>'): 'q1',
    ('q1', '<'): 'q1',
    ('q1', '_'): 'q1',
    ('q1', 'g'): 'q1',
    ('q1', 'c'): 'q1',
    ('q1', 'J'): 'q1',
    ('q1', '9'): 'q1',
    ('q1', 's'): 'q1',
    ('q1', 'ó'): 'q1',
    ('q1', 'é'): 'q1',
    ('q1', 'O'): 'q1',
    ('q1', 'Ô'): 'q1',
    ('q1', '0'): 'q1',
    ('q1', 'F'): 'q1',
    ('q1', 'M'): 'q1',
    ('q1', '`'): 'q1',
    ('q1', ''): 'q1',
    ('q1', 'Ã'): 'q1',
    ('q1', 'H'): 'q1',
    ('q1', 'õ'): 'q1',
    ('q1', '@'): 'q1',
    ('q1', 'U'): 'q1',
    ('q1', 'b'): 'q1',
    ('q1', '['): 'q1',
    ('q1', 'ê'): 'q1',
    ('q1', '5'): 'q1',
    ('q1', 'T'): 'q1',
    ('q1', '+'): 'q1',
    ('q1', '|'): 'q1',
    ('q1', 'q'): 'q1',
    ('q1', '$'): 'q1',
    ('q1', 'p'): 'q1',
    ('q1', '('): 'q1',
    ('q1', '6'): 'q1',
    ('q1', 'ú'): 'q1',
    ('q1', '3'): 'q1',
    ('q1', 'X'): 'q1',
    ('q1', 'W'): 'q1',
    ('q1', 'Ç'): 'q1',
    ('q1', 'G'): 'q1',
    ('q1', 'Q'): 'q1',
    ('q1', '-'): 'q1',
    ('q1', ':'): 'q1',
    ('q1', 'R'): 'q1',
    ('q1', 'k'): 'q1',
    ('q1', 'A'): 'q1',
    ('q1', '^'): 'q1',
    ('q1', '}'): 'q1',
    ('q1', '8'): 'q1',
    ('q1', 'Á'): 'q1',
    ('q1', 'e'): 'q1',
    ('q1', 'I'): 'q1',
    ('q1', 'a'): 'q1',
    ('q1', 'Ú'): 'q1',
    ('q1', 'Â'): 'q1',
    ('q1', 'É'): 'q1',
    ('q1', 't'): 'q1',
    ('q1', 'K'): 'q1',
    ('q1', 'â'): 'q1',
    ('q1', '*'): 'q1',
    ('q1', '~'): 'q1',
    ('q1', 'h'): 'q1',
    ('q1', 'l'): 'q1',
    ('q1', 'f'): 'q1',
    ('q1', '\\'): 'q1',
    ('q1', 'y'): 'q1',
    ('q1', 'j'): 'q1',
    ('q1', 'd'): 'q1',
    ('q1', 'L'): 'q1',
    ('q1', 'Õ'): 'q1',
    ('q1', 'Ó'): 'q1',
    ('q1', 'z'): 'q1',
    ('q1', 'C'): 'q1',
    ('q1', 'o'): 'q1',
    ('q2', 'C'): 'q3', #Recebendo C avança para o estado q3
    ('q2', 'c'): 'q3', #Recebendo c avança para o estado q3
    ('q2', 'S'): 'q1',
    ('q2', 'ç'): 'q1',
    ('q2', ','): 'q1',
    ('q2', 'N'): 'q1',
    ('q2', 'm'): 'q1',
    ('q2', '&'): 'q1',
    ('q2', 'Í'): 'q1',
    ('q2', 'Ê'): 'q1',
    ('q2', 'D'): 'q1',
    ('q2', 'B'): 'q1',
    ('q2', 'à'): 'q1',
    ('q2', 'ô'): 'q1',
    ('q2', '"'): 'q1',
    ('q2', '{'): 'q1',
    ('q2', '%'): 'q1',
    ('q2', 'á'): 'q1',
    ('q2', 'À'): 'q1',
    ('q2', 'E'): 'q1',
    ('q2', '?'): 'q1',
    ('q2', 'Z'): 'q1',
    ('q2', 'x'): 'q1',
    ('q2', '7'): 'q1',
    ('q2', '4'): 'q1',
    ('q2', ']'): 'q1',
    ('q2', ';'): 'q1',
    ('q2', 'r'): 'q1',
    ('q2', '/'): 'q1',
    ('q2', '!'): 'q1',
    ('q2', 'P'): 'q1',
    ('q2', '='): 'q1',
    ('q2', ' '): 'q2', #Recebendo espaço permanece no estado q2
    ('q2', 'í'): 'q1',
    ('q2', 'i'): 'q1',
    ('q2', 'ã'): 'q1',
    ('q2', 'u'): 'q1',
    ('q2', 'V'): 'q1',
    ('q2', '.'): 'q1',
    ('q2', '1'): 'q1',
    ('q2', "'"): 'q1',
    ('q2', 'v'): 'q1',
    ('q2', '2'): 'q1',
    ('q2', 'Y'): 'q1',
    ('q2', '#'): 'q1',
    ('q2', 'n'): 'q1',
    ('q2', ')'): 'q1',
    ('q2', 'w'): 'q1',
    ('q2', '>'): 'q1',
    ('q2', '<'): 'q1',
    ('q2', '_'): 'q1',
    ('q2', 'g'): 'q1',
    ('q2', 'J'): 'q1',
    ('q2', '9'): 'q1',
    ('q2', 's'): 'q1',
    ('q2', 'ó'): 'q1',
    ('q2', 'é'): 'q1',
    ('q2', 'O'): 'q1',
    ('q2', 'Ô'): 'q1',
    ('q2', '0'): 'q1',
    ('q2', 'F'): 'q1',
    ('q2', 'M'): 'q1',
    ('q2', '`'): 'q1',
    ('q2', ''): 'q1',
    ('q2', 'Ã'): 'q1',
    ('q2', 'H'): 'q1',
    ('q2', 'õ'): 'q1',
    ('q2', '@'): 'q1',
    ('q2', 'U'): 'q1',
    ('q2', 'b'): 'q1',
    ('q2', '['): 'q1',
    ('q2', 'ê'): 'q1',
    ('q2', '5'): 'q1',
    ('q2', 'T'): 'q1',
    ('q2', '+'): 'q1',
    ('q2', '|'): 'q1',
    ('q2', 'q'): 'q1',
    ('q2', '$'): 'q1',
    ('q2', 'p'): 'q1',
    ('q2', '('): 'q1',
    ('q2', '6'): 'q1',
    ('q2', 'ú'): 'q1',
    ('q2', '3'): 'q1',
    ('q2', 'X'): 'q1',
    ('q2', 'W'): 'q1',
    ('q2', 'Ç'): 'q1',
    ('q2', 'G'): 'q1',
    ('q2', 'Q'): 'q1',
    ('q2', '-'): 'q1',
    ('q2', ':'): 'q1',
    ('q2', 'R'): 'q1',
    ('q2', 'k'): 'q1',
    ('q2', 'A'): 'q1',
    ('q2', '^'): 'q1',
    ('q2', '}'): 'q1',
    ('q2', '8'): 'q1',
    ('q2', 'Á'): 'q1',
    ('q2', 'e'): 'q1',
    ('q2', 'I'): 'q1',
    ('q2', 'a'): 'q1',
    ('q2', 'Ú'): 'q1',
    ('q2', 'Â'): 'q1',
    ('q2', 'É'): 'q1',
    ('q2', 't'): 'q1',
    ('q2', 'K'): 'q1',
    ('q2', 'â'): 'q1',
    ('q2', '*'): 'q1',
    ('q2', '~'): 'q1',
    ('q2', 'h'): 'q1',
    ('q2', 'l'): 'q1',
    ('q2', 'f'): 'q1',
    ('q2', '\\'): 'q1',
    ('q2', 'y'): 'q1',
    ('q2', 'j'): 'q1',
    ('q2', 'd'): 'q1',
    ('q2', 'L'): 'q1',
    ('q2', 'Õ'): 'q1',
    ('q2', 'Ó'): 'q1',
    ('q2', 'z'): 'q1',
    ('q2', 'o'): 'q1',
    ('q3', 'o'): 'q4', #q3 recebe o avança para q4
    ('q3', 'S'): 'q1',
    ('q3', 'ç'): 'q1',
    ('q3', ','): 'q1',
    ('q3', 'N'): 'q1',
    ('q3', 'm'): 'q1',
    ('q3', '&'): 'q1',
    ('q3', 'Í'): 'q1',
    ('q3', 'Ê'): 'q1',
    ('q3', 'D'): 'q1',
    ('q3', 'B'): 'q1',
    ('q3', 'à'): 'q1',
    ('q3', 'ô'): 'q1',
    ('q3', '"'): 'q1',
    ('q3', '{'): 'q1',
    ('q3', '%'): 'q1',
    ('q3', 'á'): 'q1',
    ('q3', 'À'): 'q1',
    ('q3', 'E'): 'q1',
    ('q3', '?'): 'q1',
    ('q3', 'Z'): 'q1',
    ('q3', 'x'): 'q1',
    ('q3', '7'): 'q1',
    ('q3', '4'): 'q1',
    ('q3', ']'): 'q1',
    ('q3', ';'): 'q1',
    ('q3', 'r'): 'q1',
    ('q3', '/'): 'q1',
    ('q3', '!'): 'q1',
    ('q3', 'P'): 'q1',
    ('q3', '='): 'q1',
    ('q3', ' '): 'q2', #q3 recebe espaço retorna para q2
    ('q3', 'í'): 'q1',
    ('q3', 'i'): 'q1',
    ('q3', 'ã'): 'q1',
    ('q3', 'u'): 'q1',
    ('q3', 'V'): 'q1',
    ('q3', '.'): 'q1',
    ('q3', '1'): 'q1',
    ('q3', "'"): 'q1',
    ('q3', 'v'): 'q1',
    ('q3', '2'): 'q1',
    ('q3', 'Y'): 'q1',
    ('q3', '#'): 'q1',
    ('q3', 'n'): 'q1',
    ('q3', ')'): 'q1',
    ('q3', 'w'): 'q1',
    ('q3', '>'): 'q1',
    ('q3', '<'): 'q1',
    ('q3', '_'): 'q1',
    ('q3', 'g'): 'q1',
    ('q3', 'c'): 'q1',
    ('q3', 'J'): 'q1',
    ('q3', '9'): 'q1',
    ('q3', 's'): 'q1',
    ('q3', 'ó'): 'q1',
    ('q3', 'é'): 'q1',
    ('q3', 'O'): 'q1',
    ('q3', 'Ô'): 'q1',
    ('q3', '0'): 'q1',
    ('q3', 'F'): 'q1',
    ('q3', 'M'): 'q1',
    ('q3', '`'): 'q1',
    ('q3', ''): 'q1',
    ('q3', 'Ã'): 'q1',
    ('q3', 'H'): 'q1',
    ('q3', 'õ'): 'q1',
    ('q3', '@'): 'q1',
    ('q3', 'U'): 'q1',
    ('q3', 'b'): 'q1',
    ('q3', '['): 'q1',
    ('q3', 'ê'): 'q1',
    ('q3', '5'): 'q1',
    ('q3', 'T'): 'q1',
    ('q3', '+'): 'q1',
    ('q3', '|'): 'q1',
    ('q3', 'q'): 'q1',
    ('q3', '$'): 'q1',
    ('q3', 'p'): 'q1',
    ('q3', '('): 'q1',
    ('q3', '6'): 'q1',
    ('q3', 'ú'): 'q1',
    ('q3', '3'): 'q1',
    ('q3', 'X'): 'q1',
    ('q3', 'W'): 'q1',
    ('q3', 'Ç'): 'q1',
    ('q3', 'G'): 'q1',
    ('q3', 'Q'): 'q1',
    ('q3', '-'): 'q1',
    ('q3', ':'): 'q1',
    ('q3', 'R'): 'q1',
    ('q3', 'k'): 'q1',
    ('q3', 'A'): 'q1',
    ('q3', '^'): 'q1',
    ('q3', '}'): 'q1',
    ('q3', '8'): 'q1',
    ('q3', 'Á'): 'q1',
    ('q3', 'e'): 'q1',
    ('q3', 'I'): 'q1',
    ('q3', 'a'): 'q1',
    ('q3', 'Ú'): 'q1',
    ('q3', 'Â'): 'q1',
    ('q3', 'É'): 'q1',
    ('q3', 't'): 'q1',
    ('q3', 'K'): 'q1',
    ('q3', 'â'): 'q1',
    ('q3', '*'): 'q1',
    ('q3', '~'): 'q1',
    ('q3', 'h'): 'q1',
    ('q3', 'l'): 'q1',
    ('q3', 'f'): 'q1',
    ('q3', '\\'): 'q1',
    ('q3', 'y'): 'q1',
    ('q3', 'j'): 'q1',
    ('q3', 'd'): 'q1',
    ('q3', 'L'): 'q1',
    ('q3', 'Õ'): 'q1',
    ('q3', 'Ó'): 'q1',
    ('q3', 'z'): 'q1',
    ('q3', 'C'): 'q1',
    ('q4', 'm'): 'q5', # q4 recebe m avança para q5
    ('q4', 'S'): 'q1',
    ('q4', 'ç'): 'q1',
    ('q4', ','): 'q1',
    ('q4', 'N'): 'q1',
    ('q4', '&'): 'q1',
    ('q4', 'Í'): 'q1',
    ('q4', 'Ê'): 'q1',
    ('q4', 'D'): 'q1',
    ('q4', 'B'): 'q1',
    ('q4', 'à'): 'q1',
    ('q4', 'ô'): 'q1',
    ('q4', '"'): 'q1',
    ('q4', '{'): 'q1',
    ('q4', '%'): 'q1',
    ('q4', 'á'): 'q1',
    ('q4', 'À'): 'q1',
    ('q4', 'E'): 'q1',
    ('q4', '?'): 'q1',
    ('q4', 'Z'): 'q1',
    ('q4', 'x'): 'q1',
    ('q4', '7'): 'q1',
    ('q4', '4'): 'q1',
    ('q4', ']'): 'q1',
    ('q4', ';'): 'q1',
    ('q4', 'r'): 'q1',
    ('q4', '/'): 'q1',
    ('q4', '!'): 'q1',
    ('q4', 'P'): 'q1',
    ('q4', '='): 'q1',
    ('q4', ' '): 'q2', # q4 recebe espaço volta para q2
    ('q4', 'í'): 'q1',
    ('q4', 'i'): 'q1',
    ('q4', 'ã'): 'q1',
    ('q4', 'u'): 'q1',
    ('q4', 'V'): 'q1',
    ('q4', '.'): 'q1',
    ('q4', '1'): 'q1',
    ('q4', "'"): 'q1',
    ('q4', 'v'): 'q1',
    ('q4', '2'): 'q1',
    ('q4', 'Y'): 'q1',
    ('q4', '#'): 'q1',
    ('q4', 'n'): 'q1',
    ('q4', ')'): 'q1',
    ('q4', 'w'): 'q1',
    ('q4', '>'): 'q1',
    ('q4', '<'): 'q1',
    ('q4', '_'): 'q1',
    ('q4', 'g'): 'q1',
    ('q4', 'c'): 'q1',
    ('q4', 'J'): 'q1',
    ('q4', '9'): 'q1',
    ('q4', 's'): 'q1',
    ('q4', 'ó'): 'q1',
    ('q4', 'é'): 'q1',
    ('q4', 'O'): 'q1',
    ('q4', 'Ô'): 'q1',
    ('q4', '0'): 'q1',
    ('q4', 'F'): 'q1',
    ('q4', 'M'): 'q1',
    ('q4', '`'): 'q1',
    ('q4', ''): 'q1',
    ('q4', 'Ã'): 'q1',
    ('q4', 'H'): 'q1',
    ('q4', 'õ'): 'q1',
    ('q4', '@'): 'q1',
    ('q4', 'U'): 'q1',
    ('q4', 'b'): 'q1',
    ('q4', '['): 'q1',
    ('q4', 'ê'): 'q1',
    ('q4', '5'): 'q1',
    ('q4', 'T'): 'q1',
    ('q4', '+'): 'q1',
    ('q4', '|'): 'q1',
    ('q4', 'q'): 'q1',
    ('q4', '$'): 'q1',
    ('q4', 'p'): 'q1',
    ('q4', '('): 'q1',
    ('q4', '6'): 'q1',
    ('q4', 'ú'): 'q1',
    ('q4', '3'): 'q1',
    ('q4', 'X'): 'q1',
    ('q4', 'W'): 'q1',
    ('q4', 'Ç'): 'q1',
    ('q4', 'G'): 'q1',
    ('q4', 'Q'): 'q1',
    ('q4', '-'): 'q1',
    ('q4', ':'): 'q1',
    ('q4', 'R'): 'q1',
    ('q4', 'k'): 'q1',
    ('q4', 'A'): 'q1',
    ('q4', '^'): 'q1',
    ('q4', '}'): 'q1',
    ('q4', '8'): 'q1',
    ('q4', 'Á'): 'q1',
    ('q4', 'e'): 'q1',
    ('q4', 'I'): 'q1',
    ('q4', 'a'): 'q1',
    ('q4', 'Ú'): 'q1',
    ('q4', 'Â'): 'q1',
    ('q4', 'É'): 'q1',
    ('q4', 't'): 'q1',
    ('q4', 'K'): 'q1',
    ('q4', 'â'): 'q1',
    ('q4', '*'): 'q1',
    ('q4', '~'): 'q1',
    ('q4', 'h'): 'q1',
    ('q4', 'l'): 'q1',
    ('q4', 'f'): 'q1',
    ('q4', '\\'): 'q1',
    ('q4', 'y'): 'q1',
    ('q4', 'j'): 'q1',
    ('q4', 'd'): 'q1',
    ('q4', 'L'): 'q1',
    ('q4', 'Õ'): 'q1',
    ('q4', 'Ó'): 'q1',
    ('q4', 'z'): 'q1',
    ('q4', 'C'): 'q1',
    ('q4', 'o'): 'q1',
    ('q5', 'p'): 'q6', # q5 recebe p avança para q6
    ('q5', 'S'): 'q1',
    ('q5', 'ç'): 'q1',
    ('q5', ','): 'q1',
    ('q5', 'N'): 'q1',
    ('q5', 'm'): 'q1',
    ('q5', '&'): 'q1',
    ('q5', 'Í'): 'q1',
    ('q5', 'Ê'): 'q1',
    ('q5', 'D'): 'q1',
    ('q5', 'B'): 'q1',
    ('q5', 'à'): 'q1',
    ('q5', 'ô'): 'q1',
    ('q5', '"'): 'q1',
    ('q5', '{'): 'q1',
    ('q5', '%'): 'q1',
    ('q5', 'á'): 'q1',
    ('q5', 'À'): 'q1',
    ('q5', 'E'): 'q1',
    ('q5', '?'): 'q1',
    ('q5', 'Z'): 'q1',
    ('q5', 'x'): 'q1',
    ('q5', '7'): 'q1',
    ('q5', '4'): 'q1',
    ('q5', ']'): 'q1',
    ('q5', ';'): 'q1',
    ('q5', 'r'): 'q1',
    ('q5', '/'): 'q1',
    ('q5', '!'): 'q1',
    ('q5', 'P'): 'q1',
    ('q5', '='): 'q1',
    ('q5', ' '): 'q2', # q5 recebe espaço volta para q2
    ('q5', 'í'): 'q1',
    ('q5', 'i'): 'q1',
    ('q5', 'ã'): 'q1',
    ('q5', 'u'): 'q1',
    ('q5', 'V'): 'q1',
    ('q5', '.'): 'q1',
    ('q5', '1'): 'q1',
    ('q5', "'"): 'q1',
    ('q5', 'v'): 'q1',
    ('q5', '2'): 'q1',
    ('q5', 'Y'): 'q1',
    ('q5', '#'): 'q1',
    ('q5', 'n'): 'q1',
    ('q5', ')'): 'q1',
    ('q5', 'w'): 'q1',
    ('q5', '>'): 'q1',
    ('q5', '<'): 'q1',
    ('q5', '_'): 'q1',
    ('q5', 'g'): 'q1',
    ('q5', 'c'): 'q1',
    ('q5', 'J'): 'q1',
    ('q5', '9'): 'q1',
    ('q5', 's'): 'q1',
    ('q5', 'ó'): 'q1',
    ('q5', 'é'): 'q1',
    ('q5', 'O'): 'q1',
    ('q5', 'Ô'): 'q1',
    ('q5', '0'): 'q1',
    ('q5', 'F'): 'q1',
    ('q5', 'M'): 'q1',
    ('q5', '`'): 'q1',
    ('q5', ''): 'q1',
    ('q5', 'Ã'): 'q1',
    ('q5', 'H'): 'q1',
    ('q5', 'õ'): 'q1',
    ('q5', '@'): 'q1',
    ('q5', 'U'): 'q1',
    ('q5', 'b'): 'q1',
    ('q5', '['): 'q1',
    ('q5', 'ê'): 'q1',
    ('q5', '5'): 'q1',
    ('q5', 'T'): 'q1',
    ('q5', '+'): 'q1',
    ('q5', '|'): 'q1',
    ('q5', 'q'): 'q1',
    ('q5', '$'): 'q1',
    ('q5', '('): 'q1',
    ('q5', '6'): 'q1',
    ('q5', 'ú'): 'q1',
    ('q5', '3'): 'q1',
    ('q5', 'X'): 'q1',
    ('q5', 'W'): 'q1',
    ('q5', 'Ç'): 'q1',
    ('q5', 'G'): 'q1',
    ('q5', 'Q'): 'q1',
    ('q5', '-'): 'q1',
    ('q5', ':'): 'q1',
    ('q5', 'R'): 'q1',
    ('q5', 'k'): 'q1',
    ('q5', 'A'): 'q1',
    ('q5', '^'): 'q1',
    ('q5', '}'): 'q1',
    ('q5', '8'): 'q1',
    ('q5', 'Á'): 'q1',
    ('q5', 'e'): 'q1',
    ('q5', 'I'): 'q1',
    ('q5', 'a'): 'q1',
    ('q5', 'Ú'): 'q1',
    ('q5', 'Â'): 'q1',
    ('q5', 'É'): 'q1',
    ('q5', 't'): 'q1',
    ('q5', 'K'): 'q1',
    ('q5', 'â'): 'q1',
    ('q5', '*'): 'q1',
    ('q5', '~'): 'q1',
    ('q5', 'h'): 'q1',
    ('q5', 'l'): 'q1',
    ('q5', 'f'): 'q1',
    ('q5', '\\'): 'q1',
    ('q5', 'y'): 'q1',
    ('q5', 'j'): 'q1',
    ('q5', 'd'): 'q1',
    ('q5', 'L'): 'q1',
    ('q5', 'Õ'): 'q1',
    ('q5', 'Ó'): 'q1',
    ('q5', 'z'): 'q1',
    ('q5', 'C'): 'q1',
    ('q5', 'o'): 'q1',
    ('q6', 'u'): 'q7', # q6 recebe u avança para q7
    ('q6', 'S'): 'q1',
    ('q6', 'ç'): 'q1',
    ('q6', ','): 'q1',
    ('q6', 'N'): 'q1',
    ('q6', 'm'): 'q1',
    ('q6', '&'): 'q1',
    ('q6', 'Í'): 'q1',
    ('q6', 'Ê'): 'q1',
    ('q6', 'D'): 'q1',
    ('q6', 'B'): 'q1',
    ('q6', 'à'): 'q1',
    ('q6', 'ô'): 'q1',
    ('q6', '"'): 'q1',
    ('q6', '{'): 'q1',
    ('q6', '%'): 'q1',
    ('q6', 'á'): 'q1',
    ('q6', 'À'): 'q1',
    ('q6', 'E'): 'q1',
    ('q6', '?'): 'q1',
    ('q6', 'Z'): 'q1',
    ('q6', 'x'): 'q1',
    ('q6', '7'): 'q1',
    ('q6', '4'): 'q1',
    ('q6', ']'): 'q1',
    ('q6', ';'): 'q1',
    ('q6', 'r'): 'q1',
    ('q6', '/'): 'q1',
    ('q6', '!'): 'q1',
    ('q6', 'P'): 'q1',
    ('q6', '='): 'q1',
    ('q6', ' '): 'q2', # q6 recebe espaço volta para q2
    ('q6', 'í'): 'q1',
    ('q6', 'i'): 'q1',
    ('q6', 'ã'): 'q1',
    ('q6', 'V'): 'q1',
    ('q6', '.'): 'q1',
    ('q6', '1'): 'q1',
    ('q6', "'"): 'q1',
    ('q6', 'v'): 'q1',
    ('q6', '2'): 'q1',
    ('q6', 'Y'): 'q1',
    ('q6', '#'): 'q1',
    ('q6', 'n'): 'q1',
    ('q6', ')'): 'q1',
    ('q6', 'w'): 'q1',
    ('q6', '>'): 'q1',
    ('q6', '<'): 'q1',
    ('q6', '_'): 'q1',
    ('q6', 'g'): 'q1',
    ('q6', 'c'): 'q1',
    ('q6', 'J'): 'q1',
    ('q6', '9'): 'q1',
    ('q6', 's'): 'q1',
    ('q6', 'ó'): 'q1',
    ('q6', 'é'): 'q1',
    ('q6', 'O'): 'q1',
    ('q6', 'Ô'): 'q1',
    ('q6', '0'): 'q1',
    ('q6', 'F'): 'q1',
    ('q6', 'M'): 'q1',
    ('q6', '`'): 'q1',
    ('q6', ''): 'q1',
    ('q6', 'Ã'): 'q1',
    ('q6', 'H'): 'q1',
    ('q6', 'õ'): 'q1',
    ('q6', '@'): 'q1',
    ('q6', 'U'): 'q1',
    ('q6', 'b'): 'q1',
    ('q6', '['): 'q1',
    ('q6', 'ê'): 'q1',
    ('q6', '5'): 'q1',
    ('q6', 'T'): 'q1',
    ('q6', '+'): 'q1',
    ('q6', '|'): 'q1',
    ('q6', 'q'): 'q1',
    ('q6', '$'): 'q1',
    ('q6', 'p'): 'q1',
    ('q6', '('): 'q1',
    ('q6', '6'): 'q1',
    ('q6', 'ú'): 'q1',
    ('q6', '3'): 'q1',
    ('q6', 'X'): 'q1',
    ('q6', 'W'): 'q1',
    ('q6', 'Ç'): 'q1',
    ('q6', 'G'): 'q1',
    ('q6', 'Q'): 'q1',
    ('q6', '-'): 'q1',
    ('q6', ':'): 'q1',
    ('q6', 'R'): 'q1',
    ('q6', 'k'): 'q1',
    ('q6', 'A'): 'q1',
    ('q6', '^'): 'q1',
    ('q6', '}'): 'q1',
    ('q6', '8'): 'q1',
    ('q6', 'Á'): 'q1',
    ('q6', 'e'): 'q1',
    ('q6', 'I'): 'q1',
    ('q6', 'a'): 'q1',
    ('q6', 'Ú'): 'q1',
    ('q6', 'Â'): 'q1',
    ('q6', 'É'): 'q1',
    ('q6', 't'): 'q1',
    ('q6', 'K'): 'q1',
    ('q6', 'â'): 'q1',
    ('q6', '*'): 'q1',
    ('q6', '~'): 'q1',
    ('q6', 'h'): 'q1',
    ('q6', 'l'): 'q1',
    ('q6', 'f'): 'q1',
    ('q6', '\\'): 'q1',
    ('q6', 'y'): 'q1',
    ('q6', 'j'): 'q1',
    ('q6', 'd'): 'q1',
    ('q6', 'L'): 'q1',
    ('q6', 'Õ'): 'q1',
    ('q6', 'Ó'): 'q1',
    ('q6', 'z'): 'q1',
    ('q6', 'C'): 'q1',
    ('q6', 'o'): 'q1',
    ('q7', 't'): 'q8', # q7 recebe t avança para q8
    ('q7', 'S'): 'q1',
    ('q7', 'ç'): 'q1',
    ('q7', ','): 'q1',
    ('q7', 'N'): 'q1',
    ('q7', 'm'): 'q1',
    ('q7', '&'): 'q1',
    ('q7', 'Í'): 'q1',
    ('q7', 'Ê'): 'q1',
    ('q7', 'D'): 'q1',
    ('q7', 'B'): 'q1',
    ('q7', 'à'): 'q1',
    ('q7', 'ô'): 'q1',
    ('q7', '"'): 'q1',
    ('q7', '{'): 'q1',
    ('q7', '%'): 'q1',
    ('q7', 'á'): 'q1',
    ('q7', 'À'): 'q1',
    ('q7', 'E'): 'q1',
    ('q7', '?'): 'q1',
    ('q7', 'Z'): 'q1',
    ('q7', 'x'): 'q1',
    ('q7', '7'): 'q1',
    ('q7', '4'): 'q1',
    ('q7', ']'): 'q1',
    ('q7', ';'): 'q1',
    ('q7', 'r'): 'q1',
    ('q7', '/'): 'q1',
    ('q7', '!'): 'q1',
    ('q7', 'P'): 'q1',
    ('q7', '='): 'q1',
    ('q7', ' '): 'q2', # q7 recebe espaço volta para q2
    ('q7', 'í'): 'q1',
    ('q7', 'i'): 'q1',
    ('q7', 'ã'): 'q1',
    ('q7', 'u'): 'q1',
    ('q7', 'V'): 'q1',
    ('q7', '.'): 'q1',
    ('q7', '1'): 'q1',
    ('q7', "'"): 'q1',
    ('q7', 'v'): 'q1',
    ('q7', '2'): 'q1',
    ('q7', 'Y'): 'q1',
    ('q7', '#'): 'q1',
    ('q7', 'n'): 'q1',
    ('q7', ')'): 'q1',
    ('q7', 'w'): 'q1',
    ('q7', '>'): 'q1',
    ('q7', '<'): 'q1',
    ('q7', '_'): 'q1',
    ('q7', 'g'): 'q1',
    ('q7', 'c'): 'q1',
    ('q7', 'J'): 'q1',
    ('q7', '9'): 'q1',
    ('q7', 's'): 'q1',
    ('q7', 'ó'): 'q1',
    ('q7', 'é'): 'q1',
    ('q7', 'O'): 'q1',
    ('q7', 'Ô'): 'q1',
    ('q7', '0'): 'q1',
    ('q7', 'F'): 'q1',
    ('q7', 'M'): 'q1',
    ('q7', '`'): 'q1',
    ('q7', ''): 'q1',
    ('q7', 'Ã'): 'q1',
    ('q7', 'H'): 'q1',
    ('q7', 'õ'): 'q1',
    ('q7', '@'): 'q1',
    ('q7', 'U'): 'q1',
    ('q7', 'b'): 'q1',
    ('q7', '['): 'q1',
    ('q7', 'ê'): 'q1',
    ('q7', '5'): 'q1',
    ('q7', 'T'): 'q1',
    ('q7', '+'): 'q1',
    ('q7', '|'): 'q1',
    ('q7', 'q'): 'q1',
    ('q7', '$'): 'q1',
    ('q7', 'p'): 'q1',
    ('q7', '('): 'q1',
    ('q7', '6'): 'q1',
    ('q7', 'ú'): 'q1',
    ('q7', '3'): 'q1',
    ('q7', 'X'): 'q1',
    ('q7', 'W'): 'q1',
    ('q7', 'Ç'): 'q1',
    ('q7', 'G'): 'q1',
    ('q7', 'Q'): 'q1',
    ('q7', '-'): 'q1',
    ('q7', ':'): 'q1',
    ('q7', 'R'): 'q1',
    ('q7', 'k'): 'q1',
    ('q7', 'A'): 'q1',
    ('q7', '^'): 'q1',
    ('q7', '}'): 'q1',
    ('q7', '8'): 'q1',
    ('q7', 'Á'): 'q1',
    ('q7', 'e'): 'q1',
    ('q7', 'I'): 'q1',
    ('q7', 'a'): 'q1',
    ('q7', 'Ú'): 'q1',
    ('q7', 'Â'): 'q1',
    ('q7', 'É'): 'q1',
    ('q7', 'K'): 'q1',
    ('q7', 'â'): 'q1',
    ('q7', '*'): 'q1',
    ('q7', '~'): 'q1',
    ('q7', 'h'): 'q1',
    ('q7', 'l'): 'q1',
    ('q7', 'f'): 'q1',
    ('q7', '\\'): 'q1',
    ('q7', 'y'): 'q1',
    ('q7', 'j'): 'q1',
    ('q7', 'd'): 'q1',
    ('q7', 'L'): 'q1',
    ('q7', 'Õ'): 'q1',
    ('q7', 'Ó'): 'q1',
    ('q7', 'z'): 'q1',
    ('q7', 'C'): 'q1',
    ('q7', 'o'): 'q1',
    ('q8', 'a'): 'q9', # q8 recebe a e avança para q9
    ('q8', 'S'): 'q1',
    ('q8', 'ç'): 'q1',
    ('q8', ','): 'q1',
    ('q8', 'N'): 'q1',
    ('q8', 'm'): 'q1',
    ('q8', '&'): 'q1',
    ('q8', 'Í'): 'q1',
    ('q8', 'Ê'): 'q1',
    ('q8', 'D'): 'q1',
    ('q8', 'B'): 'q1',
    ('q8', 'à'): 'q1',
    ('q8', 'ô'): 'q1',
    ('q8', '"'): 'q1',
    ('q8', '{'): 'q1',
    ('q8', '%'): 'q1',
    ('q8', 'á'): 'q1',
    ('q8', 'À'): 'q1',
    ('q8', 'E'): 'q1',
    ('q8', '?'): 'q1',
    ('q8', 'Z'): 'q1',
    ('q8', 'x'): 'q1',
    ('q8', '7'): 'q1',
    ('q8', '4'): 'q1',
    ('q8', ']'): 'q1',
    ('q8', ';'): 'q1',
    ('q8', 'r'): 'q1',
    ('q8', '/'): 'q1',
    ('q8', '!'): 'q1',
    ('q8', 'P'): 'q1',
    ('q8', '='): 'q1',
    ('q8', ' '): 'q2', # q8 recebe espaço e volta para q2
    ('q8', 'í'): 'q1',
    ('q8', 'i'): 'q1',
    ('q8', 'ã'): 'q1',
    ('q8', 'u'): 'q1',
    ('q8', 'V'): 'q1',
    ('q8', '.'): 'q1',
    ('q8', '1'): 'q1',
    ('q8', "'"): 'q1',
    ('q8', 'v'): 'q1',
    ('q8', '2'): 'q1',
    ('q8', 'Y'): 'q1',
    ('q8', '#'): 'q1',
    ('q8', 'n'): 'q1',
    ('q8', ')'): 'q1',
    ('q8', 'w'): 'q1',
    ('q8', '>'): 'q1',
    ('q8', '<'): 'q1',
    ('q8', '_'): 'q1',
    ('q8', 'g'): 'q1',
    ('q8', 'c'): 'q1',
    ('q8', 'J'): 'q1',
    ('q8', '9'): 'q1',
    ('q8', 's'): 'q1',
    ('q8', 'ó'): 'q1',
    ('q8', 'é'): 'q1',
    ('q8', 'O'): 'q1',
    ('q8', 'Ô'): 'q1',
    ('q8', '0'): 'q1',
    ('q8', 'F'): 'q1',
    ('q8', 'M'): 'q1',
    ('q8', '`'): 'q1',
    ('q8', ''): 'q1',
    ('q8', 'Ã'): 'q1',
    ('q8', 'H'): 'q1',
    ('q8', 'õ'): 'q1',
    ('q8', '@'): 'q1',
    ('q8', 'U'): 'q1',
    ('q8', 'b'): 'q1',
    ('q8', '['): 'q1',
    ('q8', 'ê'): 'q1',
    ('q8', '5'): 'q1',
    ('q8', 'T'): 'q1',
    ('q8', '+'): 'q1',
    ('q8', '|'): 'q1',
    ('q8', 'q'): 'q1',
    ('q8', '$'): 'q1',
    ('q8', 'p'): 'q1',
    ('q8', '('): 'q1',
    ('q8', '6'): 'q1',
    ('q8', 'ú'): 'q1',
    ('q8', '3'): 'q1',
    ('q8', 'X'): 'q1',
    ('q8', 'W'): 'q1',
    ('q8', 'Ç'): 'q1',
    ('q8', 'G'): 'q1',
    ('q8', 'Q'): 'q1',
    ('q8', '-'): 'q1',
    ('q8', ':'): 'q1',
    ('q8', 'R'): 'q1',
    ('q8', 'k'): 'q1',
    ('q8', 'A'): 'q1',
    ('q8', '^'): 'q1',
    ('q8', '}'): 'q1',
    ('q8', '8'): 'q1',
    ('q8', 'Á'): 'q1',
    ('q8', 'e'): 'q1',
    ('q8', 'I'): 'q1',
    ('q8', 'Ú'): 'q1',
    ('q8', 'Â'): 'q1',
    ('q8', 'É'): 'q1',
    ('q8', 't'): 'q1',
    ('q8', 'K'): 'q1',
    ('q8', 'â'): 'q1',
    ('q8', '*'): 'q1',
    ('q8', '~'): 'q1',
    ('q8', 'h'): 'q1',
    ('q8', 'l'): 'q1',
    ('q8', 'f'): 'q1',
    ('q8', '\\'): 'q1',
    ('q8', 'y'): 'q1',
    ('q8', 'j'): 'q1',
    ('q8', 'd'): 'q1',
    ('q8', 'L'): 'q1',
    ('q8', 'Õ'): 'q1',
    ('q8', 'Ó'): 'q1',
    ('q8', 'z'): 'q1',
    ('q8', 'C'): 'q1',
    ('q8', 'o'): 'q1',
    ('q9', 'd'): 'q10', # q9 recebe d avança para q10
    ('q9', 'S'): 'q1',
    ('q9', 'ç'): 'q1',
    ('q9', ','): 'q1',
    ('q9', 'N'): 'q1',
    ('q9', 'm'): 'q1',
    ('q9', '&'): 'q1',
    ('q9', 'Í'): 'q1',
    ('q9', 'Ê'): 'q1',
    ('q9', 'D'): 'q1',
    ('q9', 'B'): 'q1',
    ('q9', 'à'): 'q1',
    ('q9', 'ô'): 'q1',
    ('q9', '"'): 'q1',
    ('q9', '{'): 'q1',
    ('q9', '%'): 'q1',
    ('q9', 'á'): 'q1',
    ('q9', 'À'): 'q1',
    ('q9', 'E'): 'q1',
    ('q9', '?'): 'q1',
    ('q9', 'Z'): 'q1',
    ('q9', 'x'): 'q1',
    ('q9', '7'): 'q1',
    ('q9', '4'): 'q1',
    ('q9', ']'): 'q1',
    ('q9', ';'): 'q1',
    ('q9', 'r'): 'q1',
    ('q9', '/'): 'q1',
    ('q9', '!'): 'q1',
    ('q9', 'P'): 'q1',
    ('q9', '='): 'q1',
    ('q9', ' '): 'q2', # q9 recebe espaço volta para q2
    ('q9', 'í'): 'q1',
    ('q9', 'i'): 'q1',
    ('q9', 'ã'): 'q1',
    ('q9', 'u'): 'q1',
    ('q9', 'V'): 'q1',
    ('q9', '.'): 'q1',
    ('q9', '1'): 'q1',
    ('q9', "'"): 'q1',
    ('q9', 'v'): 'q1',
    ('q9', '2'): 'q1',
    ('q9', 'Y'): 'q1',
    ('q9', '#'): 'q1',
    ('q9', 'n'): 'q1',
    ('q9', ')'): 'q1',
    ('q9', 'w'): 'q1',
    ('q9', '>'): 'q1',
    ('q9', '<'): 'q1',
    ('q9', '_'): 'q1',
    ('q9', 'g'): 'q1',
    ('q9', 'c'): 'q1',
    ('q9', 'J'): 'q1',
    ('q9', '9'): 'q1',
    ('q9', 's'): 'q1',
    ('q9', 'ó'): 'q1',
    ('q9', 'é'): 'q1',
    ('q9', 'O'): 'q1',
    ('q9', 'Ô'): 'q1',
    ('q9', '0'): 'q1',
    ('q9', 'F'): 'q1',
    ('q9', 'M'): 'q1',
    ('q9', '`'): 'q1',
    ('q9', ''): 'q1',
    ('q9', 'Ã'): 'q1',
    ('q9', 'H'): 'q1',
    ('q9', 'õ'): 'q1',
    ('q9', '@'): 'q1',
    ('q9', 'U'): 'q1',
    ('q9', 'b'): 'q1',
    ('q9', '['): 'q1',
    ('q9', 'ê'): 'q1',
    ('q9', '5'): 'q1',
    ('q9', 'T'): 'q1',
    ('q9', '+'): 'q1',
    ('q9', '|'): 'q1',
    ('q9', 'q'): 'q1',
    ('q9', '$'): 'q1',
    ('q9', 'p'): 'q1',
    ('q9', '('): 'q1',
    ('q9', '6'): 'q1',
    ('q9', 'ú'): 'q1',
    ('q9', '3'): 'q1',
    ('q9', 'X'): 'q1',
    ('q9', 'W'): 'q1',
    ('q9', 'Ç'): 'q1',
    ('q9', 'G'): 'q1',
    ('q9', 'Q'): 'q1',
    ('q9', '-'): 'q1',
    ('q9', ':'): 'q1',
    ('q9', 'R'): 'q1',
    ('q9', 'k'): 'q1',
    ('q9', 'A'): 'q1',
    ('q9', '^'): 'q1',
    ('q9', '}'): 'q1',
    ('q9', '8'): 'q1',
    ('q9', 'Á'): 'q1',
    ('q9', 'e'): 'q1',
    ('q9', 'I'): 'q1',
    ('q9', 'a'): 'q1',
    ('q9', 'Ú'): 'q1',
    ('q9', 'Â'): 'q1',
    ('q9', 'É'): 'q1',
    ('q9', 't'): 'q1',
    ('q9', 'K'): 'q1',
    ('q9', 'â'): 'q1',
    ('q9', '*'): 'q1',
    ('q9', '~'): 'q1',
    ('q9', 'h'): 'q1',
    ('q9', 'l'): 'q1',
    ('q9', 'f'): 'q1',
    ('q9', '\\'): 'q1',
    ('q9', 'y'): 'q1',
    ('q9', 'j'): 'q1',
    ('q9', 'L'): 'q1',
    ('q9', 'Õ'): 'q1',
    ('q9', 'Ó'): 'q1',
    ('q9', 'z'): 'q1',
    ('q9', 'C'): 'q1',
    ('q9', 'o'): 'q1',
    ('q10', 'o'): 'q11', # q10 recebe o avança para q11
    ('q10', 'S'): 'q1',
    ('q10', 'ç'): 'q1',
    ('q10', ','): 'q1',
    ('q10', 'N'): 'q1',
    ('q10', 'm'): 'q1',
    ('q10', '&'): 'q1',
    ('q10', 'Í'): 'q1',
    ('q10', 'Ê'): 'q1',
    ('q10', 'D'): 'q1',
    ('q10', 'B'): 'q1',
    ('q10', 'à'): 'q1',
    ('q10', 'ô'): 'q1',
    ('q10', '"'): 'q1',
    ('q10', '{'): 'q1',
    ('q10', '%'): 'q1',
    ('q10', 'á'): 'q1',
    ('q10', 'À'): 'q1',
    ('q10', 'E'): 'q1',
    ('q10', '?'): 'q1',
    ('q10', 'Z'): 'q1',
    ('q10', 'x'): 'q1',
    ('q10', '7'): 'q1',
    ('q10', '4'): 'q1',
    ('q10', ']'): 'q1',
    ('q10', ';'): 'q1',
    ('q10', 'r'): 'q1',
    ('q10', '/'): 'q1',
    ('q10', '!'): 'q1',
    ('q10', 'P'): 'q1',
    ('q10', '='): 'q1',
    ('q10', ' '): 'q2', # q10 recebe o volta para q2
    ('q10', 'í'): 'q1',
    ('q10', 'i'): 'q1',
    ('q10', 'ã'): 'q1',
    ('q10', 'u'): 'q1',
    ('q10', 'V'): 'q1',
    ('q10', '.'): 'q1',
    ('q10', '1'): 'q1',
    ('q10', "'"): 'q1',
    ('q10', 'v'): 'q1',
    ('q10', '2'): 'q1',
    ('q10', 'Y'): 'q1',
    ('q10', '#'): 'q1',
    ('q10', 'n'): 'q1',
    ('q10', ')'): 'q1',
    ('q10', 'w'): 'q1',
    ('q10', '>'): 'q1',
    ('q10', '<'): 'q1',
    ('q10', '_'): 'q1',
    ('q10', 'g'): 'q1',
    ('q10', 'c'): 'q1',
    ('q10', 'J'): 'q1',
    ('q10', '9'): 'q1',
    ('q10', 's'): 'q1',
    ('q10', 'ó'): 'q1',
    ('q10', 'é'): 'q1',
    ('q10', 'O'): 'q1',
    ('q10', 'Ô'): 'q1',
    ('q10', '0'): 'q1',
    ('q10', 'F'): 'q1',
    ('q10', 'M'): 'q1',
    ('q10', '`'): 'q1',
    ('q10', ''): 'q1',
    ('q10', 'Ã'): 'q1',
    ('q10', 'H'): 'q1',
    ('q10', 'õ'): 'q1',
    ('q10', '@'): 'q1',
    ('q10', 'U'): 'q1',
    ('q10', 'b'): 'q1',
    ('q10', '['): 'q1',
    ('q10', 'ê'): 'q1',
    ('q10', '5'): 'q1',
    ('q10', 'T'): 'q1',
    ('q10', '+'): 'q1',
    ('q10', '|'): 'q1',
    ('q10', 'q'): 'q1',
    ('q10', '$'): 'q1',
    ('q10', 'p'): 'q1',
    ('q10', '('): 'q1',
    ('q10', '6'): 'q1',
    ('q10', 'ú'): 'q1',
    ('q10', '3'): 'q1',
    ('q10', 'X'): 'q1',
    ('q10', 'W'): 'q1',
    ('q10', 'Ç'): 'q1',
    ('q10', 'G'): 'q1',
    ('q10', 'Q'): 'q1',
    ('q10', '-'): 'q1',
    ('q10', ':'): 'q1',
    ('q10', 'R'): 'q1',
    ('q10', 'k'): 'q1',
    ('q10', 'A'): 'q1',
    ('q10', '^'): 'q1',
    ('q10', '}'): 'q1',
    ('q10', '8'): 'q1',
    ('q10', 'Á'): 'q1',
    ('q10', 'e'): 'q1',
    ('q10', 'I'): 'q1',
    ('q10', 'a'): 'q1',
    ('q10', 'Ú'): 'q1',
    ('q10', 'Â'): 'q1',
    ('q10', 'É'): 'q1',
    ('q10', 't'): 'q1',
    ('q10', 'K'): 'q1',
    ('q10', 'â'): 'q1',
    ('q10', '*'): 'q1',
    ('q10', '~'): 'q1',
    ('q10', 'h'): 'q1',
    ('q10', 'l'): 'q1',
    ('q10', 'f'): 'q1',
    ('q10', '\\'): 'q1',
    ('q10', 'y'): 'q1',
    ('q10', 'j'): 'q1',
    ('q10', 'd'): 'q1',
    ('q10', 'L'): 'q1',
    ('q10', 'Õ'): 'q1',
    ('q10', 'Ó'): 'q1',
    ('q10', 'z'): 'q1',
    ('q10', 'C'): 'q1',
    ('q11', 'r'): 'q12', # q11 recebe r avança para q12
    ('q11', 'S'): 'q1', 
    ('q11', 'ç'): 'q1',
    ('q11', ','): 'q1',
    ('q11', 'N'): 'q1',
    ('q11', 'm'): 'q1',
    ('q11', '&'): 'q1',
    ('q11', 'Í'): 'q1',
    ('q11', 'Ê'): 'q1',
    ('q11', 'D'): 'q1',
    ('q11', 'B'): 'q1',
    ('q11', 'à'): 'q1',
    ('q11', 'ô'): 'q1',
    ('q11', '"'): 'q1',
    ('q11', '{'): 'q1',
    ('q11', '%'): 'q1',
    ('q11', 'á'): 'q1',
    ('q11', 'À'): 'q1',
    ('q11', 'E'): 'q1',
    ('q11', '?'): 'q1',
    ('q11', 'Z'): 'q1',
    ('q11', 'x'): 'q1',
    ('q11', '7'): 'q1',
    ('q11', '4'): 'q1',
    ('q11', ']'): 'q1',
    ('q11', ';'): 'q1',
    ('q11', '/'): 'q1',
    ('q11', '!'): 'q1',
    ('q11', 'P'): 'q1',
    ('q11', '='): 'q1',
    ('q11', ' '): 'q2', # q11 recebe espaço volta para q2
    ('q11', 'í'): 'q1',
    ('q11', 'i'): 'q1',
    ('q11', 'ã'): 'q1',
    ('q11', 'u'): 'q1',
    ('q11', 'V'): 'q1',
    ('q11', '.'): 'q1',
    ('q11', '1'): 'q1',
    ('q11', "'"): 'q1',
    ('q11', 'v'): 'q1',
    ('q11', '2'): 'q1',
    ('q11', 'Y'): 'q1',
    ('q11', '#'): 'q1',
    ('q11', 'n'): 'q1',
    ('q11', ')'): 'q1',
    ('q11', 'w'): 'q1',
    ('q11', '>'): 'q1',
    ('q11', '<'): 'q1',
    ('q11', '_'): 'q1',
    ('q11', 'g'): 'q1',
    ('q11', 'c'): 'q1',
    ('q11', 'J'): 'q1',
    ('q11', '9'): 'q1',
    ('q11', 's'): 'q1',
    ('q11', 'ó'): 'q1',
    ('q11', 'é'): 'q1',
    ('q11', 'O'): 'q1',
    ('q11', 'Ô'): 'q1',
    ('q11', '0'): 'q1',
    ('q11', 'F'): 'q1',
    ('q11', 'M'): 'q1',
    ('q11', '`'): 'q1',
    ('q11', ''): 'q1',
    ('q11', 'Ã'): 'q1',
    ('q11', 'H'): 'q1',
    ('q11', 'õ'): 'q1',
    ('q11', '@'): 'q1',
    ('q11', 'U'): 'q1',
    ('q11', 'b'): 'q1',
    ('q11', '['): 'q1',
    ('q11', 'ê'): 'q1',
    ('q11', '5'): 'q1',
    ('q11', 'T'): 'q1',
    ('q11', '+'): 'q1',
    ('q11', '|'): 'q1',
    ('q11', 'q'): 'q1',
    ('q11', '$'): 'q1',
    ('q11', 'p'): 'q1',
    ('q11', '('): 'q1',
    ('q11', '6'): 'q1',
    ('q11', 'ú'): 'q1',
    ('q11', '3'): 'q1',
    ('q11', 'X'): 'q1',
    ('q11', 'W'): 'q1',
    ('q11', 'Ç'): 'q1',
    ('q11', 'G'): 'q1',
    ('q11', 'Q'): 'q1',
    ('q11', '-'): 'q1',
    ('q11', ':'): 'q1',
    ('q11', 'R'): 'q1',
    ('q11', 'k'): 'q1',
    ('q11', 'A'): 'q1',
    ('q11', '^'): 'q1',
    ('q11', '}'): 'q1',
    ('q11', '8'): 'q1',
    ('q11', 'Á'): 'q1',
    ('q11', 'e'): 'q1',
    ('q11', 'I'): 'q1',
    ('q11', 'a'): 'q1',
    ('q11', 'Ú'): 'q1',
    ('q11', 'Â'): 'q1',
    ('q11', 'É'): 'q1',
    ('q11', 't'): 'q1',
    ('q11', 'K'): 'q1',
    ('q11', 'â'): 'q1',
    ('q11', '*'): 'q1',
    ('q11', '~'): 'q1',
    ('q11', 'h'): 'q1',
    ('q11', 'l'): 'q1',
    ('q11', 'f'): 'q1',
    ('q11', '\\'): 'q1',
    ('q11', 'y'): 'q1',
    ('q11', 'j'): 'q1',
    ('q11', 'd'): 'q1',
    ('q11', 'L'): 'q1',
    ('q11', 'Õ'): 'q1',
    ('q11', 'Ó'): 'q1',
    ('q11', 'z'): 'q1',
    ('q11', 'C'): 'q1',
    ('q11', 'o'): 'q1',
    ('q12', 'S'): 'q1',
    ('q12', 'ç'): 'q1',
    ('q12', ','): 'q13', # q12 recebe virgula avança para q13
    ('q12', 'N'): 'q1',
    ('q12', 'm'): 'q1',
    ('q12', '&'): 'q1',
    ('q12', 'Í'): 'q1',
    ('q12', 'Ê'): 'q1',
    ('q12', 'D'): 'q1',
    ('q12', 'B'): 'q1',
    ('q12', 'à'): 'q1',
    ('q12', 'ô'): 'q1',
    ('q12', '"'): 'q1',
    ('q12', '{'): 'q1',
    ('q12', '%'): 'q1',
    ('q12', 'á'): 'q1',
    ('q12', 'À'): 'q1',
    ('q12', 'E'): 'q1',
    ('q12', '?'): 'q13', # q12 recebe interrogação avança para q13
    ('q12', 'Z'): 'q1',
    ('q12', 'x'): 'q1',
    ('q12', '7'): 'q1',
    ('q12', '4'): 'q1',
    ('q12', ']'): 'q1',
    ('q12', ';'): 'q13', # q12 recebe ponto e virgula avança para q13
    ('q12', 'r'): 'q1',
    ('q12', '/'): 'q1',
    ('q12', '!'): 'q13', # q12 recebe exclamação avança para q13
    ('q12', 'P'): 'q1',
    ('q12', '='): 'q1',
    ('q12', ' '): 'q13', # q12 recebe espaço avança para q13
    ('q12', 'í'): 'q1',
    ('q12', 'i'): 'q1',
    ('q12', 'ã'): 'q1',
    ('q12', 'u'): 'q1',
    ('q12', 'V'): 'q1',
    ('q12', '.'): 'q13', # q12 recebe ponto avança para q13
    ('q12', '1'): 'q1',
    ('q12', "'"): 'q1',
    ('q12', 'v'): 'q1',
    ('q12', '2'): 'q1',
    ('q12', 'Y'): 'q1',
    ('q12', '#'): 'q1',
    ('q12', 'n'): 'q1',
    ('q12', ')'): 'q1',
    ('q12', 'w'): 'q1',
    ('q12', '>'): 'q1',
    ('q12', '<'): 'q1',
    ('q12', '_'): 'q1',
    ('q12', 'g'): 'q1',
    ('q12', 'c'): 'q1',
    ('q12', 'J'): 'q1',
    ('q12', '9'): 'q1',
    ('q12', 's'): 'q1',
    ('q12', 'ó'): 'q1',
    ('q12', 'é'): 'q1',
    ('q12', 'O'): 'q1',
    ('q12', 'Ô'): 'q1',
    ('q12', '0'): 'q1',
    ('q12', 'F'): 'q1',
    ('q12', 'M'): 'q1',
    ('q12', '`'): 'q1',
    ('q12', ''): 'q1',
    ('q12', 'Ã'): 'q1',
    ('q12', 'H'): 'q1',
    ('q12', 'õ'): 'q1',
    ('q12', '@'): 'q1',
    ('q12', 'U'): 'q1',
    ('q12', 'b'): 'q1',
    ('q12', '['): 'q1',
    ('q12', 'ê'): 'q1',
    ('q12', '5'): 'q1',
    ('q12', 'T'): 'q1',
    ('q12', '+'): 'q1',
    ('q12', '|'): 'q1',
    ('q12', 'q'): 'q1',
    ('q12', '$'): 'q1',
    ('q12', 'p'): 'q1',
    ('q12', '('): 'q1',
    ('q12', '6'): 'q1',
    ('q12', 'ú'): 'q1',
    ('q12', '3'): 'q1',
    ('q12', 'X'): 'q1',
    ('q12', 'W'): 'q1',
    ('q12', 'Ç'): 'q1',
    ('q12', 'G'): 'q1',
    ('q12', 'Q'): 'q1',
    ('q12', '-'): 'q1',
    ('q12', ':'): 'q13', # q12 recebe dois pontos avança para q13
    ('q12', 'R'): 'q1',
    ('q12', 'k'): 'q1',
    ('q12', 'A'): 'q1',
    ('q12', '^'): 'q1',
    ('q12', '}'): 'q1',
    ('q12', '8'): 'q1',
    ('q12', 'Á'): 'q1',
    ('q12', 'e'): 'q1',
    ('q12', 'I'): 'q1',
    ('q12', 'a'): 'q1',
    ('q12', 'Ú'): 'q1',
    ('q12', 'Â'): 'q1',
    ('q12', 'É'): 'q1',
    ('q12', 't'): 'q1',
    ('q12', 'K'): 'q1',
    ('q12', 'â'): 'q1',
    ('q12', '*'): 'q1',
    ('q12', '~'): 'q1',
    ('q12', 'h'): 'q1',
    ('q12', 'l'): 'q1',
    ('q12', 'f'): 'q1',
    ('q12', '\\'): 'q1',
    ('q12', 'y'): 'q1',
    ('q12', 'j'): 'q1',
    ('q12', 'd'): 'q1',
    ('q12', 'L'): 'q1',
    ('q12', 'Õ'): 'q1',
    ('q12', 'Ó'): 'q1',
    ('q12', 'z'): 'q1',
    ('q12', 'C'): 'q1',
    ('q12', 'o'): 'q1',
    ('q13', 'c'): 'q3', # q13 recebe c volta para q3
    ('q13', 'C'): 'q3', # q13 recebe C volta para q3
    ('q13', ' '): 'q2', # q13 recebe espaço volta para q2
    ('q13', 'S'): 'q1',
    ('q13', 'ç'): 'q1',
    ('q13', ','): 'q1',
    ('q13', 'N'): 'q1',
    ('q13', 'm'): 'q1',
    ('q13', '&'): 'q1',
    ('q13', 'Í'): 'q1',
    ('q13', 'Ê'): 'q1',
    ('q13', 'D'): 'q1',
    ('q13', 'B'): 'q1',
    ('q13', 'à'): 'q1',
    ('q13', 'ô'): 'q1',
    ('q13', '"'): 'q1',
    ('q13', '{'): 'q1',
    ('q13', '%'): 'q1',
    ('q13', 'á'): 'q1',
    ('q13', 'À'): 'q1',
    ('q13', 'E'): 'q1',
    ('q13', '?'): 'q1',
    ('q13', 'Z'): 'q1',
    ('q13', 'x'): 'q1',
    ('q13', '7'): 'q1',
    ('q13', '4'): 'q1',
    ('q13', ']'): 'q1',
    ('q13', ';'): 'q1',
    ('q13', 'r'): 'q1',
    ('q13', '/'): 'q1',
    ('q13', '!'): 'q1',
    ('q13', 'P'): 'q1',
    ('q13', '='): 'q1',
    ('q13', 'í'): 'q1',
    ('q13', 'i'): 'q1',
    ('q13', 'ã'): 'q1',
    ('q13', 'u'): 'q1',
    ('q13', 'V'): 'q1',
    ('q13', '.'): 'q1',
    ('q13', '1'): 'q1',
    ('q13', "'"): 'q1',
    ('q13', 'v'): 'q1',
    ('q13', '2'): 'q1',
    ('q13', 'Y'): 'q1',
    ('q13', '#'): 'q1',
    ('q13', 'n'): 'q1',
    ('q13', ')'): 'q1',
    ('q13', 'w'): 'q1',
    ('q13', '>'): 'q1',
    ('q13', '<'): 'q1',
    ('q13', '_'): 'q1',
    ('q13', 'g'): 'q1',
    ('q13', 'J'): 'q1',
    ('q13', '9'): 'q1',
    ('q13', 's'): 'q1',
    ('q13', 'ó'): 'q1',
    ('q13', 'é'): 'q1',
    ('q13', 'O'): 'q1',
    ('q13', 'Ô'): 'q1',
    ('q13', '0'): 'q1',
    ('q13', 'F'): 'q1',
    ('q13', 'M'): 'q1',
    ('q13', '`'): 'q1',
    ('q13', ''): 'q1',
    ('q13', 'Ã'): 'q1',
    ('q13', 'H'): 'q1',
    ('q13', 'õ'): 'q1',
    ('q13', '@'): 'q1',
    ('q13', 'U'): 'q1',
    ('q13', 'b'): 'q1',
    ('q13', '['): 'q1',
    ('q13', 'ê'): 'q1',
    ('q13', '5'): 'q1',
    ('q13', 'T'): 'q1',
    ('q13', '+'): 'q1',
    ('q13', '|'): 'q1',
    ('q13', 'q'): 'q1',
    ('q13', '$'): 'q1',
    ('q13', 'p'): 'q1',
    ('q13', '('): 'q1',
    ('q13', '6'): 'q1',
    ('q13', 'ú'): 'q1',
    ('q13', '3'): 'q1',
    ('q13', 'X'): 'q1',
    ('q13', 'W'): 'q1',
    ('q13', 'Ç'): 'q1',
    ('q13', 'G'): 'q1',
    ('q13', 'Q'): 'q1',
    ('q13', '-'): 'q1',
    ('q13', ':'): 'q1',
    ('q13', 'R'): 'q1',
    ('q13', 'k'): 'q1',
    ('q13', 'A'): 'q1',
    ('q13', '^'): 'q1',
    ('q13', '}'): 'q1',
    ('q13', '8'): 'q1',
    ('q13', 'Á'): 'q1',
    ('q13', 'e'): 'q1',
    ('q13', 'I'): 'q1',
    ('q13', 'a'): 'q1',
    ('q13', 'Ú'): 'q1',
    ('q13', 'Â'): 'q1',
    ('q13', 'É'): 'q1',
    ('q13', 't'): 'q1',
    ('q13', 'K'): 'q1',
    ('q13', 'â'): 'q1',
    ('q13', '*'): 'q1',
    ('q13', '~'): 'q1',
    ('q13', 'h'): 'q1',
    ('q13', 'l'): 'q1',
    ('q13', 'f'): 'q1',
    ('q13', '\\'): 'q1',
    ('q13', 'y'): 'q1',
    ('q13', 'j'): 'q1',
    ('q13', 'd'): 'q1',
    ('q13', 'L'): 'q1',
    ('q13', 'Õ'): 'q1',
    ('q13', 'Ó'): 'q1',
    ('q13', 'z'): 'q1',
    ('q13', 'o'): 'q1'
}